# 6. Resultados. Teste do melhor modelo

### Instalar/Importar bibliotecas

In [15]:
pip install pydantic

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import os
import numpy as np

from pydantic import BaseModel
from typing import Optional, List

import joblib

### Carregar dados de entrada

Classe para ler dados de balanco patrimonial e demonstrativo financeiro

In [17]:
class AquisicaoDadosFundamentos(BaseModel):
    balancos_dir: Optional[str] = "dados/"
    balancos_joblib_file: Optional[str] = "dados/fundamentos_teste.joblib"
    fund_by_code: Optional[dict] = {}
    codes: List[str] = []

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.codes = self.get_code_list()

    def run(self):
        if os.path.isfile(self.balancos_joblib_file):
            print("carregando joblib fundamentos")
            return joblib.load(self.balancos_joblib_file)

        self.get_balancos_by_code()
        self.get_dre_by_code()

        self.salvar_joblib()
        
        return self.fund_by_code
    
    def get_code_list(self):
        return [file.replace("balanco_", "").replace(".xls", "") for file in os.listdir(self.balancos_dir)]

    def get_balancos_by_code(self) -> None:

        # Lista os arquivos xls na pasta dados/balancos
        files = os.listdir(self.balancos_dir)

        # Para cada arquivo (empresa) de balanco guarda no dicionario com o codigo como chave
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            print(code)
            balanco = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=0)
            # colocar codigo na posicao 0, 0
            balanco.iloc[0, 0] = code
            # mudar coluna
            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]
            # tornar a 1ª coluna (que agora tem o nome da empresa)
            balanco = balanco.set_index(code)
            self.fund_by_code[code] = balanco

    def get_dre_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            dre = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=1)
            # na primeira coluna colocar o título com o nome da empresa
            dre.iloc[0, 0] = code
            # pegar 1ª linha e tornar um cabeçalho
            dre.columns = dre.iloc[0]
            dre = dre[1:]
            # tornar a 1ª coluna (que agora tem o nome da emrpesa)
            dre = dre.set_index(code)
            self.fund_by_code[code] = self.fund_by_code[code].append(dre)
    
    def salvar_joblib(self):
        joblib.dump(self.fund_by_code, self.balancos_joblib_file)



## Pré tratamento e transformacao dos dados

### Funcoes para de transformacao de dados

In [22]:
def transpor_colunas(df):
    df = df.copy()
    
    df = df.T
    df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
    
    return df

In [23]:
def rename_columns(df):
    df = df.copy()
    columns_rename={
        "Receita Líquida de Vendas e/ou Serviços": "receita_liq", 
        "Lucro/Prejuízo do Período": "lucro_op", 
        "Outros Ativos Circulantes": "outros_ativos_circ", 
        "Despesas Gerais e Administrativas": "despesas_gerais", 
        "Resultado Antes Tributação/Participações": "resultado_antes_trib"}
    
    for col, _ in columns_rename.items():
        if col not in list(df.columns):
            raise ValueError(f"coluna '{col}' nao presente no df.\n {[col for col in list(df.columns)]}")

    df.rename(columns=columns_rename, inplace=True)
            
    df.iloc[:,:-1] = df.iloc[:,:-1].astype(float)
    return df

In [24]:
def filtrar_variaveis_para_modelo(df, manter_variaveis = None):
    manter_variaveis = ['outros_ativos_circ', 'receita_liq', 'despesas_gerais', 'resultado_antes_trib', 'lucro_op']
    df = df.loc[:, manter_variaveis]

    return df

In [21]:
def transformar_indic_em_porcentagem_delta(dfs_in):
    dfs = dfs_in.copy()

    for code in dfs.keys():
        fundamento = dfs[code]
        fundamento.sort_index()
        for coluna in fundamento.columns:
            if coluna != "decisao":
                condicoes = [
                    (fundamento[coluna].shift(1) > 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] > 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] > 0),
                    (fundamento[coluna].shift(1) == 0) & (fundamento[coluna] < 0),
                    (fundamento[coluna].shift(1) < 0) & (fundamento[coluna] == 0),
                ]
                valores = [
                    -1,
                    1,
                    (abs(fundamento[coluna].shift(1)) - abs(fundamento[coluna]))
                                / abs(fundamento[coluna].shift(1)),
                    1,
                    -1,
                    1,
                ]
                fundamento[coluna] = (
                    np.select(condicoes, valores, default=fundamento[coluna]
                              / fundamento[coluna].shift(1) - 1)
                )
        dfs[code] = fundamento
    return dfs

### Predicao com melhor modelo

In [25]:
def carregar_modelo_ml():
    file_name: str = "../../out/result_ml_best_model.joblib"
    model = joblib.load(file_name)
    return model

In [26]:
def predict_decisao(modelo, fundamentos):
    for code, df in fundamentos.items():
        X = df.iloc[1:,:]
        y_predicted = modelo.predict(X)
        valor_pred = f"{y_predicted[0]}".replace('0', "Vender").replace('1', "Comprar")
        
        print(f"Decisao recomendada pelo modelo para: {code} -> {valor_pred}")

# Execucao

In [31]:
def main() -> None:
    """
        Aquisicao dos novos dados para teste
    """

    aquisicao_fund = AquisicaoDadosFundamentos()
    fundamentos_by_code = aquisicao_fund.run()

    """
        Tratamento e transformacao dos dados
    """
    for code, df in fundamentos_by_code.items():
        df_ = transpor_colunas(df)
        df_ = rename_columns(df_)
        df_ = filtrar_variaveis_para_modelo(df_)
        fundamentos_by_code[code] = df_

    fundamentos_by_code = transformar_indic_em_porcentagem_delta(fundamentos_by_code)

    """
        Predicao com melhor modelo
    """
    modelo = carregar_modelo_ml()

    predict_decisao(modelo, fundamentos_by_code)

main()

carregando joblib fundamentos
Decisao recomendada pelo modelo para: CVCB3 -> Vender
Decisao recomendada pelo modelo para: ABEV3 -> Comprar
Decisao recomendada pelo modelo para: COGN3 -> Comprar
Decisao recomendada pelo modelo para: MOVI3 -> Comprar


/Users/shopee/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
